<h1 style="background-color:green;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
    🌐 תהליך הסבת חלקות שומא לבנק"ל מודרני 🌐

In [ ]:
print('3. Tax Assessment Parcels:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from configs import CNFG
import numpy as np
import pandas as pd
import geopandas as gpd
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  3.a Reading inputs')

<h1 style="text-align:right;font-size:100%">
שכבת חלקות שומא קיימת

In [ ]:
parcels_shuma_cols = ['PARCEL', 'GUSH_NUM', 'GUSH_SUFFIX', 'LEGAL_AREA', 'geometry']

parcels_shuma = gpd.read_file(CNFG.CADSDE + 'PARCEL_ALL_SHUMA.gdb',
                              layer          = 'PARCEL_ALL_SHUMA',
                              include_fields = parcels_shuma_cols)


del parcels_shuma_cols

<h1 style="text-align:right;font-size:100%">
שכבת גושי שומא מודרנית

In [ ]:
TaxAssasmentBlocks_cols = ['BlockUniqueID', 'BlockNumber', 'SubBlockNumber']

TaxAssasmentBlocks = gpd.read_file(CNFG.ProductPath,
                                   layer = 'TaxAssessmentBlocks',
                                   include_fields = TaxAssasmentBlocks_cols,
                                   ignore_geometry = True)


del TaxAssasmentBlocks_cols

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  3.b Cleaning data')

In [ ]:
print('  3.c Data conversion')

<h1 style="text-align:right;font-size:125%;">
מיזוג חלקות שומא עם נתוני גושי שומא לקבלת מספר זיהוי של גוש השומא

In [ ]:
parcels_shuma['GUSH_NUM'] = parcels_shuma['GUSH_NUM'].astype(int)

parcels_shuma['GUSH_SUFFIX'] = parcels_shuma['GUSH_SUFFIX'].fillna(0)
parcels_shuma['GUSH_SUFFIX'] = parcels_shuma['GUSH_SUFFIX'].astype(int)

parcels_shuma['BlockName'] = parcels_shuma['GUSH_NUM'].astype(str) + '_' + parcels_shuma['GUSH_SUFFIX'].astype(str)
parcels_shuma.rename(columns = {'GUSH_NUM' : 'BlockNumber', 'GUSH_SUFFIX' : 'SubBlockNumber'}, inplace = True)

In [ ]:
TaxAssasmentBlocks['BlockNumber'] = TaxAssasmentBlocks['BlockNumber'].astype(int)
TaxAssasmentBlocks['SubBlockNumber'] = TaxAssasmentBlocks['SubBlockNumber'].astype(int)

TaxAssasmentBlocks['BlockName'] = TaxAssasmentBlocks['BlockNumber'].astype(str) + '_' + TaxAssasmentBlocks['SubBlockNumber'].astype(str)
TaxAssasmentBlocks.drop(columns = ['BlockNumber', 'SubBlockNumber'], inplace = True)

In [ ]:
parcels_shuma = parcels_shuma.merge(TaxAssasmentBlocks, on = 'BlockName', how = 'left')

del TaxAssasmentBlocks

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('  3.d Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("ParcelUniqueID", int),
                        ("ParcelNumber", int),
                        ("BlockNumber", int),
                        ("SubBlockNumber", int),
                        ("ParcelType", int),
                        ("LegalArea", float),
                        ("BlockUniqueID", int),
                        ("geometry", gpd.array.GeometryDtype)
                        ])


TaxAssessmentParcels = gpd.GeoDataFrame(data = np.empty(0, dtype=new_columns), crs = 2039, geometry = 'geometry')

del new_columns

<h1 style="text-align:right;font-size:100%">
איכלוס שדות

In [ ]:
TaxAssessmentParcels['ParcelNumber'] = parcels_shuma['PARCEL'].astype(int)

TaxAssessmentParcels['BlockNumber'] = parcels_shuma['BlockNumber'].astype(int)

TaxAssessmentParcels['SubBlockNumber'] = parcels_shuma['SubBlockNumber'].astype(int)

TaxAssessmentParcels['ParcelType'] = 3 #שומא

TaxAssessmentParcels['LegalArea'] = parcels_shuma['LEGAL_AREA'].astype(int)

TaxAssessmentParcels['BlockUniqueID'] = parcels_shuma['BlockUniqueID'].astype(int)

TaxAssessmentParcels['geometry'] = parcels_shuma['geometry']

TaxAssessmentParcels.sort_values(by = ['BlockNumber', 'ParcelNumber'], inplace = True)

TaxAssessmentParcels['ParcelUniqueID'] = np.arange(1, len(TaxAssessmentParcels) + 1)
TaxAssessmentParcels['ParcelUniqueID'] = TaxAssessmentParcels['ParcelUniqueID'].astype(int)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  3.e Exporting results to product folder')

In [ ]:
json_dir = CNFG.JsonPath + 'TaxAssessmentParcels.geojson'
TaxAssessmentParcels.to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([CNFG.BatchPath + 'Tax Assessment Parcels-Save as Feature Class.bat']);